In [1]:
from guppylang.std.builtins import comptime
from guppylang.std.quantum import qubit, measure_array
from hugr.qsystem.result import QsysResult
from guppylang.std.builtins import result, array, exit
from guppylang import guppy
from selene_sim import build, Quest, MetricStore

from trunctaylor.qtmlib.circuits.lcu import LCUMultiplexorBox
from pytket.passes import AutoRebase
from pytket import OpType
from pytket.passes import DecomposeBoxes
from pytket.circuit import StatePreparationBox

from collections import defaultdict

import pandas as pd
import numpy as np
import scipy.special as sp

from trunctaylor.operators import ising_model

### 1D Ising model Hamiltonian

In [2]:
n_state = 4
H = ising_model(n_qubits=n_state, j=1.0, h=0.5)

We construct the LCU for
$\widetilde{H}= \frac{-i}{\left\|\alpha\right\|_{1}}\sum_{\ell}^{L}\alpha_\ell H_\ell$
using multiplexor gate synthesis. We then construct a controlled-$\widetilde{H}$ for the Select oracle of the time evolution operator.

In [3]:
H_lcu = LCUMultiplexorBox(H, n_state)
H_norm = H_lcu.l1_norm
H_lcu = LCUMultiplexorBox((-1j / H_norm) * H, n_state)
n_prep = H_lcu.n_prepare_qubits

controlled_lcu = H_lcu.qcontrol(1)
circ = controlled_lcu.get_circuit()
DecomposeBoxes().apply(circ)
rebase = AutoRebase({OpType.CX, OpType.Rz, OpType.H, OpType.CCX})
rebase.apply(circ)
qlibs_controlled_lcu = guppy.load_pytket("qlibs_controlled_lcu", circ, use_arrays=True)


@guppy.comptime
def ctrl_lcu(
    control: qubit,
    prep: array[qubit, comptime(n_prep)],
    state: array[qubit, comptime(n_state)],
) -> None:
    return qlibs_controlled_lcu([control], prep, state)

Prepare the  Taylor expansion coefficients 
$\widetilde{\beta}_k = \frac{(\tau \left\|\alpha\right\|_{1})^k}{k!}$ for $\sqrt{\frac{\widetilde{\beta}_k}{\left\|\widetilde{\beta}\right\|_1}}$.

We set $K=7$, $\tau=0.05$.


In [5]:
K = 7
tau = 0.05


def beta_coeff(tau: float, alpha: float, K: int):
    beta_k = np.zeros(2 ** int(np.ceil(np.log2(K + 1))))
    beta_k[0] = 1.0
    for k in range(1, K + 1):
        beta_k[k] = ((tau * alpha) ** k) / sp.factorial(k)
    return beta_k


betas = beta_coeff(tau=tau, alpha=H_norm, K=K)
betas_rescaled = np.sqrt(betas / np.sum(np.abs(betas)))

exp_prep_box = StatePreparationBox(betas_rescaled)
circ = exp_prep_box.get_circuit()
DecomposeBoxes().apply(circ)
rebase = AutoRebase({OpType.CX, OpType.Rz, OpType.H, OpType.CCX})
rebase.apply(circ)
exp_prep = guppy.load_pytket("exp_prepare", circ, use_arrays=True)
exp_unprep = guppy.load_pytket("exp_unprepare", circ.dagger(), use_arrays=True)

Putting all together, we implement the LCU with mid-circuit measurements for the time evolution operator.
We run $10000$ shots for now.

In [6]:
n_shots = 10000

kappa = int(np.log2(K) + 1)


@guppy
def main() -> None:
    """Main function to run the multiplexor LCU circuit."""
    exp_prep_qubits = array(qubit() for _ in range(comptime(kappa)))

    state_qubits = array(qubit() for _ in range(comptime(n_state)))

    exp_prep(exp_prep_qubits)
    for kk_index in range(comptime(kappa)):
        for _ in range(2**kk_index):
            prep_qubits = array(qubit() for _ in range(comptime(n_prep)))
            ctrl_lcu(
                exp_prep_qubits[comptime(kappa) - kk_index - 1],
                prep_qubits,
                state_qubits,
            )
            outcome = measure_array(prep_qubits)
            for b in outcome:
                if b:
                    exit("circuit failed", 1)
    exp_unprep(exp_prep_qubits)
    outcome = measure_array(exp_prep_qubits)
    for b in outcome:
        if b:
            exit("circuit failed", 1)

    result("c", measure_array(state_qubits))


compiled_hugr = main.compile()
metric_store = MetricStore()

runner = build(compiled_hugr)
shots = QsysResult(
    runner.run_shots(
        Quest(random_seed=17), event_hook=metric_store, n_qubits=kappa+n_prep+n_state , n_shots=n_shots, verbose=False
    )
)

Final state and success probability

In [7]:
shots_counts = shots.register_counts()["c"]
success_prob = sum(shots_counts.values()) / n_shots

print(f'Final state: {shots_counts}')
print(f'Success probability: {success_prob}')

Final state: Counter({'0000': 6068, '1000': 5, '0001': 5, '0010': 4, '0100': 2})
Success probability: 0.6084


Circuit resources used

In [8]:
metrics = metric_store.shots
data_by_category = defaultdict(lambda: defaultdict(list))

for shot in metrics:
    for category, metric_dict in shot.items():
        for metric, value in metric_dict.items():
            data_by_category[category][metric].append(value)

headers = ["Metric"] + [f"Shot {i}" for i in range(len(metrics))]

df_user_program = pd.DataFrame(
    [[k] + v[:] for k, v in data_by_category["user_program"].items()],
    columns=headers,
)
df_user_program = df_user_program.set_index("Metric").transpose()

df_post_runtime = pd.DataFrame(
    [[k] + v[:] for k, v in data_by_category["post_runtime"].items()],
    columns=headers,
)
df_post_runtime = df_post_runtime.set_index("Metric").transpose()


print(f'Average metrics (over shots):\n{df_post_runtime.mean()}')

Average metrics (over shots):
Metric
custom_op_batch_count            0.0000
custom_op_individual_count       0.0000
measure_batch_count             23.6472
measure_individual_count        23.6472
reset_batch_count               25.6231
reset_individual_count          25.6231
rxy_batch_count               2829.5006
rxy_individual_count          2829.5006
rz_batch_count                4495.0515
rz_individual_count           4495.0515
rzz_batch_count                942.3360
rzz_individual_count           942.3360
total_duration_ns                0.0000
dtype: float64


Classical results

In [9]:
H_mat = H.to_sparse_matrix().toarray()
state = np.zeros(2**n_state)
state[0] = 1.0

p_analytical = []

for K in range(1, 8):
    betas = beta_coeff(tau=0.05, alpha=H_norm, K=K)
    exp_ihht = betas[0] * np.identity(2**n_state)
    for k in range(1, K + 1):
        Hk = H_mat / H_norm
        for kk in range(k - 1):
            Hk = Hk @ H_mat / H_norm
        exp_ihht = exp_ihht + (-1j) ** k * betas[k] * Hk
    
    statef = exp_ihht @ state

    p_analytical.append(statef.conj().T @ statef / ((sum(betas)) ** 2))
    print(f'Success probability (analytical), K={K}: {statef.conj().T @ statef / ((sum(betas)) ** 2)}')

Success probability (analytical), K=1: (0.6559999999999999+0j)
Success probability (analytical), K=2: (0.6092673408685306+0j)
Success probability (analytical), K=3: (0.6066575788750417+0j)
Success probability (analytical), K=4: (0.6065385140258427+0j)
Success probability (analytical), K=5: (0.6065310248629526+0j)
Success probability (analytical), K=6: (0.6065306716397784+0j)
Success probability (analytical), K=7: (0.6065306600635357+0j)
